# Sprint22 リカレントニューラルネットワーク

## 【問題1】SimpleRNNのフォワードプロパゲーション実装
---
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

In [2]:
import numpy as np

class SimpleInitializerRNN:
    """
    RNN用の初期化
    """
    def __init__(self, sigma):
        pass
    def Wx(self, n_features, n_nodes):
        """
        重みの初期化
        Parameters
        ----------
        n_features : int
          特徴量の数
        n_nodes : int
          ノード数
        Returns
        ----------
        W :
        """
        w = np.zeros((n_features, n_nodes))
        return w

    def Wh(self, n_nodes):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes : int
          ノード数
        Returns
        ----------
        W :
        """
        w = np.zeros((n_nodes, n_nodes))
        return w

    def B(self, n_nodes):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes : int
          ノード数
        Returns
        ----------
        B :
        """
        b = np.zeros(n_nodes)
        return b

class XavierInitializer:
    """
    Xavierによる初期化
    Parameters
    ----------
    """
    def __init__(self, sigma):
        _ = sigma

    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
        """
        sigma = 1.0 / np.sqrt(n_nodes1)
        w = sigma * np.random.randn(n_nodes1, n_nodes2)
        return w

    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
        """
        b = np.random.randn(n_nodes2)
        return b

class HeInitializer:
    """
    Heによる初期化
    Parameters
    ----------
    """
    def __init__(self, sigma):
        _ = sigma

    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
        """
        sigma = np.sqrt( 2.0 / n_nodes1)
        w = sigma * np.random.randn(n_nodes1, n_nodes2)
        return w

    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
        """
        b = np.random.randn(n_nodes2)
        return b

# 活性化関数クラス化
class Softmax:
    """
    ソフトマックス関数
    Parameters
    ----------

    """
    def __init__(self):
        self.loss = None

    def forward(self, X):

        X = X.T
        y = np.exp(X) / np.sum(np.exp(X), axis=0)

        return y.T

    def backward(self, Z3, y):
        batch_size = y.shape[0]
        ret = (Z3 - y)/batch_size

        # lossの計算
        self.loss = cross_entropy_error(y, Z3)

        return ret


class ReLU:
    """
    ReLU関数
    Parameters
    ----------
    """
    def __init__(self):
        self.x = None

    def forward(self, X):

        self.x = X

        return np.maximum(0, X)

    def backward(self, X):

        return np.where(self.x > 0, X, 0)


class Tanh:
    """
    ハイボリックタンジェント関数
    Parameters
    ----------
    """
    def __init__(self):
        self.dA = None

    def forward(self, X):

        self.dA = np.tanh(X)
        return self.dA

    def backward(self, dZ):

        return dZ*(1 - np.tanh(self.dA)**2)


# 最適化手法
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """

        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB

        return layer


class SimpleRNN:
    def __init__(self, n_features, n_nodes, optimizer, initializer):
        self.optimizer = optimizer
        self.initializer = initializer

        self.Wx = self.initializer.Wx(n_features, n_nodes)
        self.Wh = self.initializer.Wh(n_nodes)
        self.B = self.initializer.B(n_nodes)

        self.h = None
        self.x = None

        # 微分した重みとバイアス
        self.dWx = None
        self.dWh = None
        self.dB = None

    def forward(self, xs):
        Wx = self.Wx
        Wh = self.Wh
        B = self.B
        batch_size, n_sequences, n_features = xs.shape
        n_features, n_nodes = Wx.shape

        hs = np.empty((batch_size, n_sequences, n_nodes), dtype='f')

        for seq in range(n_sequences):

            x = xs[:, seq, :]
            h_prev = self.h

            # ここでRNNの計算
            t = np.dot(h_prev, Wh) + np.dot(x, Wx) + B
            h_next = np.tanh(t)

            # backwardで使用するため保持
            self.x = x
            self.h = h_next
            hs[:, seq, :] = self.h

        return hs

    def backward(self, dA):
        pass

        # 更新
        self = self.optimizer.update(self)

        return None


class ScratchSimpleRNNClassifier:
    def __init__(self, batch_size, n_features, n_nodes, lr, sigma=0.01, optimizer=SGD, activation=Tanh, initializer=SimpleInitializerRNN):
        self.batch_size = batch_size
        self.n_features = n_features
        self.n_nodes = n_nodes
        self.lr = lr        # 学習率
        self.sigma = sigma  # ガウス分布の標準偏差

        # self.optimizer = optimizer(self.lr)
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        self.RNN1 = SimpleRNN(self.n_features, self.n_nodes, self.initializer(self.sigma), self.optimizer(self.lr))
        self.activation1 = self.activation()
        self.RNN2 = SimpleRNN(self.n_features, self.n_nodes, self.initializer(self.sigma), self.optimizer(self.lr))
        self.activation2 = self.activation()
        self.RNN3 = SimpleRNN(self.n_features, self.n_nodes, self.initializer(self.sigma), self.optimizer(self.lr))
        self.activation3 = Softmax()

        self.loss_list = []

    def fit(self, X, y, X_val=None, y_val=None):

        Z3 = self._forward(X)
        self._backward(X, y, Z3)

        # 損失関数の値を保持
        self.loss_list.append(self.activation3.loss)


    def _forward(self, X):

        A1 = self.RNN1.forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.RNN2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.RNN3.forward(Z2)
        Z3 = self.activation3.forward(A3)

        return Z3


    def _backward(self, X, y, Z3):

        dA3 = self.activation3.backward(Z3, y) # 交差エントロピー誤差とソフトマックスを合わせている
        dZ2 = self.RNN3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.RNN2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dZ0 = self.RNN1.backward(dA1) # dZ0は使用しない

    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_features)
            特徴量ベクトル
        Returns
        -------
        y_pred：次の形のndarray, shape (batch_size, n_output)
            推定結果（10個の確率の中で、最も高いインデックス＝各ラベル（0〜9））
        """
        y_pred = self._forward(X)

        return y_pred.argmax(axis=1)


## 【問題2】小さな配列でのフォワードプロパゲーションの実験
---
小さな配列でフォワードプロパゲーションを考えてみます。

入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。


In [3]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [4]:
rnn = SimpleRNN(n_features, n_nodes, SGD, SimpleInitializerRNN(0.05))
rnn.Wh = w_h
rnn.Wx = w_x
rnn.B = b
rnn.h = h
hs = rnn.forward(x)
print(hs)
print(rnn.h)
# h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)


[[[0.76188797 0.76213956 0.762391   0.7625584 ]
  [0.792209   0.8141834  0.8340491  0.84977716]
  [0.79494226 0.81839    0.8393965  0.85584176]]]
[[0.79494228 0.81839002 0.83939649 0.85584174]]
